# Step 1 - Establishing the Board

In [1]:
rows = 'ABCDEFGHI'
cols = '123456789'

In [2]:
def cross(a, b):
    """
    a,b -> Strings, returns a list of strings
    cross('abc', 'def') will return ['ad', 'ae', 'af', 'bd', 'be', 'bf', 'cd', 'ce', 'cf'].
    """
    return [s+t for s in a for t in b]

In [3]:
boxes = cross(rows, cols)

In [4]:
#boxes
# Firstly form all the rows
# row_units[0] is the top most row
row_units = [cross(r, cols) for r in rows]

# Now form all the columns in column
# So column_units[0] - will return all the labels in FIRST column
column_units = [cross(rows, c) for c in cols]

# This is for the top left square - square_units[0], 3 would be middle left
square_units = [cross(rs, cs) for rs in ('ABC','DEF','GHI') for cs in ('123','456','789')]

In [5]:
# Create entire list
unitlist = row_units + column_units + square_units

# Step 2 - Form the grid_values() 
- i.e. for each dictionary connect where it is empty '.' or... it has a value

In [6]:
square_units

[['A1', 'A2', 'A3', 'B1', 'B2', 'B3', 'C1', 'C2', 'C3'],
 ['A4', 'A5', 'A6', 'B4', 'B5', 'B6', 'C4', 'C5', 'C6'],
 ['A7', 'A8', 'A9', 'B7', 'B8', 'B9', 'C7', 'C8', 'C9'],
 ['D1', 'D2', 'D3', 'E1', 'E2', 'E3', 'F1', 'F2', 'F3'],
 ['D4', 'D5', 'D6', 'E4', 'E5', 'E6', 'F4', 'F5', 'F6'],
 ['D7', 'D8', 'D9', 'E7', 'E8', 'E9', 'F7', 'F8', 'F9'],
 ['G1', 'G2', 'G3', 'H1', 'H2', 'H3', 'I1', 'I2', 'I3'],
 ['G4', 'G5', 'G6', 'H4', 'H5', 'H6', 'I4', 'I5', 'I6'],
 ['G7', 'G8', 'G9', 'H7', 'H8', 'H9', 'I7', 'I8', 'I9']]

In [7]:
def grid_values(grid):
    """Convert grid string into {<box>: <value>} dict with '.' value for empties.

    Args:
        grid: Sudoku grid in string form, 81 characters long
    Returns:
        Sudoku grid in dictionary form:
        - keys: Box labels, e.g. 'A1'
        - values: Value in corresponding box, e.g. '8', or '.' if it is empty.
    """
    assert len(grid) == 81 # must ensure that this is 81
    # https://stackoverflow.com/questions/209840/map-two-lists-into-a-dictionary-in-python
    dictionary = dict(zip(boxes, grid))
    
    # Replace all . values with '123456789'
    for keys, values in dictionary.items():
        if values == '.':
            dictionary[keys] = '123456789'
            
    
    return dictionary     

In [15]:
#grid_values('...........................999999999999123456789099999999999999119987654321099999')

# Step 3 - Elimination
- Goes through eliminating single values

In [4]:
units = dict((s, [u for u in unitlist if s in u]) for s in boxes)
peers = dict((s, set(sum(units[s],[]))-set([s])) for s in boxes)

NameError: name 'boxes' is not defined

In [10]:
def eliminate(values):
    """Eliminate values from peers of each box with a single value.

    Go through all the boxes, and whenever there is a box with a single value,
    eliminate this value from the set of values of all its peers.

    Args:
        values: Sudoku in dictionary form.
    Returns:
        Resulting Sudoku in dictionary form after eliminating values.
    """
    # Find each key or value which is len == 1, add them to a list
    solvedList = []
    for keys, v in values.items():
        if len(v) == 1:
            solvedList.append(keys)
    # print (solvedList)
        
    # Check against dictionary
    for s in solvedList:
        num = values[s]
        for p in peers[s]:
            values[p] = values[p].replace(num,'')
        
    return values

# Step 4 - Only Choice
- Go through all the units, and whenever there is a unit with a value that only fits in one box, assign the value to this box.

In [12]:
dictx = {'C2': ['2', '5', '7'], 'A1': ['1','4', '5'], 'C1': ['2', '5'], 'A2': ['4', '5', '7', '8'], 'B3': ['4', '7'], 'B2': ['2', '4', '6', '7', '8']}

In [13]:
dictx

{'A1': ['1', '4', '5'],
 'A2': ['4', '5', '7', '8'],
 'B2': ['2', '4', '6', '7', '8'],
 'B3': ['4', '7'],
 'C1': ['2', '5'],
 'C2': ['2', '5', '7']}

In [57]:
units = dict((s, [u for u in unitlist if s in u]) for s in boxes)

# Step 5 - Implement Only_choice

In [67]:
def only_choice(values):
    """Finalize all values that are the only choice for a unit.

    Go through all the units, and whenever there is a unit with a value
    that only fits in one box, assign the value to this box.

    Input: Sudoku in dictionary form.
    Output: Resulting Sudoku in dictionary form after filling in only choices.
    """
    # TODO: Implement only choice strategy here
    # So intention would be to look for each SQUARE UNIT
    for unit in unitlist:
        # This is a list of lists (each list is the ones in square)
        listOfUnchosen = []
        listOfUnchosenValues = []
        listOfUnique = []
        uniqueValues = []
        flipped = {}
        dictx = {}
        # iterate firstly over all the values to find non len 1 ones
        for s in unit:

            listOfUnchosen.append(s)
            listOfUnchosenValues.append(list(values[s]))

        dictx = dict(zip(listOfUnchosen,listOfUnchosenValues))
        # print(dictx)
        
        # Check for unique values

        for k, v in dictx.items():
            for l in v:
                if l not in flipped:
                    flipped[l] = [k]
                else:
                    flipped[l].append(k)

        for f, g in flipped.items():
            if len(g) == 1:
                for pos in g:
                    listOfUnique.append(pos)
                    uniqueValues.append(f)

        for l in listOfUnique:
            # Make sure nothing else in the list has this number!!!
            tempCheck = []
            for u in unit:
                tempCheck.append(values[u])

            #if uniqueValues[listOfUnique.index(l)] not in tempCheck:
            #print(tempCheck)
            values[l] = uniqueValues[listOfUnique.index(l)]
   
        
    return values

In [63]:
x = ((len(values[s]), s) for s in boxes if len(values[s]) > 1)

# boxes is a list of of every thing

In [64]:
((len(values[s]), s) for s in boxes) 
# so this is getting the len, and the 

<generator object <genexpr> at 0x10912d6d0>

In [69]:
import random

def search(values):
    "Using depth-first search and propagation, create a search tree and solve the sudoku."
    # First, reduce the puzzle using the previous function
    values = reduce_puzzle(values)

    if values is False:
        return False 
    if all(len(values[s]) == 1 for s in boxes): 
        return values 
        
    # Choose one of the unfilled squares with the fewest possibilities
    # Each key values.keys will have a box, so you find the length of each box
    box_lengths = [len(values[box]) for box in values.keys() if len(values[box]) > 1] 
    
    # find the lowest 
    min_box = min(box_lengths)
    
    # Find squares
    sq = [box for box in values.keys() if len(values[box]) == min_box]
    s = random.choice(sq)
   
    # Recursion to solve EACH sudoku
    # for value in values[s]
    for value in values[s]:
        new_sudoku = values.copy()
        new_sudoku[s] = value
        attempt = search(new_sudoku)
        if attempt:
            return attempt


In [6]:
rows = 'ABCDEFGHI'
cols = '123456789'

def cross(a, b):
    return [s+t for s in a for t in b]

boxes = cross(rows, cols)

row_units = [cross(r, cols) for r in rows]
column_units = [cross(rows, c) for c in cols]
square_units = [cross(rs, cs) for rs in ('ABC','DEF','GHI') for cs in ('123','456','789')]
unitlist = row_units + column_units + square_units
units = dict((s, [u for u in unitlist if s in u]) for s in boxes)
peers = dict((s, set(sum(units[s],[]))-set([s])) for s in boxes)

def display(values):
    """
    Display the values as a 2-D grid.
    Input: The sudoku in dictionary form
    Output: None
    """
    width = 1+max(len(values[s]) for s in boxes)
    line = '+'.join(['-'*(width*3)]*3)
    for r in rows:
        print(''.join(values[r+c].center(width)+('|' if c in '36' else '')
                      for c in cols))
        if r in 'CF': print(line)
    return

def grid_values(grid):
    """
    Convert grid into a dict of {square: char} with '123456789' for empties.
    Input: A grid in string form.
    Output: A grid in dictionary form
            Keys: The boxes, e.g., 'A1'
            Values: The value in each box, e.g., '8'. If the box has no value, then the value will be '123456789'.
    """
    chars = []
    digits = '123456789'
    for c in grid:
        if c in digits:
            chars.append(c)
        if c == '.':
            chars.append(digits)
    assert len(chars) == 81
    return dict(zip(boxes, chars))
    
def eliminate(values):
    """
    Go through all the boxes, and whenever there is a box with a value, eliminate this value from the values of all its peers.
    Input: A sudoku in dictionary form.
    Output: The resulting sudoku in dictionary form.
    """
    solved_values = [box for box in values.keys() if len(values[box]) == 1]
    for box in solved_values:
        digit = values[box]
        for peer in peers[box]:
            values[peer] = values[peer].replace(digit,'')
    return values

def only_choice(values):
    """
    Go through all the units, and whenever there is a unit with a value that only fits in one box, assign the value to this box.
    Input: A sudoku in dictionary form.
    Output: The resulting sudoku in dictionary form.
    """
    for unit in unitlist:
        for digit in '123456789':
            dplaces = [box for box in unit if digit in values[box]]
            if len(dplaces) == 1:
                values[dplaces[0]] = digit
    return values

def reduce_puzzle(values):
    """
    Iterate eliminate() and only_choice(). If at some point, there is a box with no available values, return False.
    If the sudoku is solved, return the sudoku.
    If after an iteration of both functions, the sudoku remains the same, return the sudoku.
    Input: A sudoku in dictionary form.
    Output: The resulting sudoku in dictionary form.
    """
    solved_values = [box for box in values.keys() if len(values[box]) == 1]
    stalled = False
    while not stalled:
        solved_values_before = len([box for box in values.keys() if len(values[box]) == 1])
        values = eliminate(values)
        values = only_choice(values)
        solved_values_after = len([box for box in values.keys() if len(values[box]) == 1])
        stalled = solved_values_before == solved_values_after
        if len([box for box in values.keys() if len(values[box]) == 0]):
            return False
    return values

In [7]:
c = '12345'

In [8]:
if '12' in c:
    print("Works")

Works


In [16]:
values = {'A1':2, 'B2':3 }
ux = ['A1','B2']

In [13]:
for v in dicx:
    print (v)

A1
B2


In [17]:
l_val=[values[s] for s in ux]

In [18]:
l_val

[2, 3]

In [21]:
u_dupes =[l_val.count(s) for s in l_val]

In [22]:
u_dupes

[1, 1]